<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/intro_analytics_python/main/images/PA%20Banner.png" width="1000" align="center"></a>


# Probabilidad y estadística

Programa creado para mostrar ejemplos prácticos de los visto durante la clase\
v1.1

In [ ]:
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
!pip install category_encoders
import category_encoders as ce

     |████████████████████████████████| 81kB 3.5MB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Ensayo de probabilidad de una moneda

In [ ]:
# Ejecutar:
if os.access('partidos.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/partidos.csv > partidos.csv
    else:
        !wget partidos.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/partidos.csv

--2021-01-07 14:12:20--  http://partidos.csv/
Resolving partidos.csv (partidos.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘partidos.csv’
--2021-01-07 14:12:20--  https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/partidos.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3143988 (3.0M) [text/plain]
Saving to: ‘partidos.csv’

partidos.csv        100%[===================>]   3.00M  --.-KB/s    in 0.1s    

2021-01-07 14:12:20 (21.7 MB/s) - ‘partidos.csv’ saved [3143988/3143988]

FINISHED --2021-01-07 14:12:20--
Total wall clock time: 0.5s
Downloaded: 1 files, 3.0M in 0.1s (21.7 MB/s)


In [ ]:
df = pd.read_csv("partidos.csv")
des = df.describe()
des.loc['Nan'] = df.isna().sum()
des.loc['%Nan'] = (df.isna().mean())*100
des

,home_score,away_score
count,41586.000000,41586.000000
mean,1.745756,1.187587
std,1.753780,1.405323
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,1.000000
75%,2.000000,2.000000
max,31.000000,21.000000
Nan,0.000000,0.000000
%Nan,0.000000,0.000000


### Análisis de la media

In [ ]:
df.describe()

,home_score,away_score
count,41586.000000,41586.000000
mean,1.745756,1.187587
std,1.753780,1.405323
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,1.000000
75%,2.000000,2.000000
max,31.000000,21.000000


In [ ]:
df.tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
41581,2020-01-10,Barbados,Canada,1,4,Friendly,Irvine,United States,True
41582,2020-01-12,Kosovo,Sweden,0,1,Friendly,Doha,Qatar,True
41583,2020-01-15,Canada,Iceland,0,1,Friendly,Irvine,United States,True
41584,2020-01-19,El Salvador,Iceland,0,1,Friendly,Carson,United States,True
41585,2020-02-01,United States,Costa Rica,1,0,Friendly,Carson,United States,False


In [ ]:
# Limpiando columnas sin importancia
df_1 = pd.DataFrame(df[['date', 'home_team', 'away_team', 'neutral']])

# Añadiendo columna "win" que determina el resultado del partido
df_1['win'] = df.apply(lambda x: 1 if x['home_score'] > x['away_score'] else 
                     0 if x['away_score'] > x['home_score'] else 0.5, axis=1)

# Si bien no existe un indicador tan especifico como conocer los jugadores con lo que 
# que cuenta cada equipo en cada partido, la fecha resulta ser un indicador, aunque un 
# poco mas impreciso, para conocer dicha informacion, si 2 equipos se enfrentan entre si,
# la fecha podria ser determinante para predecir el resultado, si uno de los equipo ha
# tenido un buen rendimiento ese mismo año, podria ser decisivo para deducir con exactitud
# si ha sido ganador o no, por lo que nos lo vamos a quedar para el analisis  
df_1['date'] = df_1.apply(lambda x: x['date'][:4], axis=1) 



# Ejecutando el BinaryEncoder
be = ce.BinaryEncoder()
paises_df = pd.concat([df_1['home_team'], df_1['away_team']])
paises = pd.DataFrame(np.sort(paises_df.unique()))
be_df = be.fit_transform(paises)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
paises.join(be_df)

,0,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8,0_9
0,Abkhazia,0,0,0,0,0,0,0,0,0,1
1,Afghanistan,0,0,0,0,0,0,0,0,1,0
2,Albania,0,0,0,0,0,0,0,0,1,1
3,Alderney,0,0,0,0,0,0,0,1,0,0
4,Algeria,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
307,Yugoslavia,0,1,0,0,1,1,0,1,0,0
308,Zambia,0,1,0,0,1,1,0,1,0,1
309,Zanzibar,0,1,0,0,1,1,0,1,1,0
310,Zimbabwe,0,1,0,0,1,1,0,1,1,1


In [ ]:
#pd.merge(left=df_1, right=paises, left_on='home_team', right_on=0)
pd.merge(df_1,paises,on='home_team',how='0')

KeyError: ignored